## Quinto caso práctico de Modelos de Simulación
#### - Rivera Juan Matías

In [146]:
import datetime

##### Función transforma un datetime en un string con valor igual al tiempo(time)

In [147]:
def transformarTiempo(tiempo):
    st = datetime.datetime.strptime(str(tiempo), "%Y-%m-%d %H:%M:%S")
    return st.time()

#### Clase de cola

In [148]:
class Cola:
    def __init__(self,q,TiempoLlegada,DeltaLlegada):
        self.q=q
        self.tiempoL=TiempoLlegada
        self.deltaL = DeltaLlegada
        self.contador = 0

    def Llegada(self,PS):
        tiempoA = self.tiempoL
        if PS.zona == 0 and PS.estado == 0:
            PS.zona = 1
            PS.tiempoZ = self.tiempoL + PS.deltaZ
        else:
            self.q += 1
        self.tiempoL += self.deltaL[self.contador]
        self.contador +=1 
        return tiempoA
    
    def ReinicioContador(self):
        if self.contador > len(self.deltaL)-1:
            self.contador = 0

#### Clase de Puesto de Servicio

In [149]:
class PuestoDeServicio:
    def __init__(self,Estado,TiempoFS,DeltaFS,DeltaZ,Limite,TiempoActual):
        self.estado = Estado
        self.tiempoFS = TiempoFS
        self.deltaFS = DeltaFS
        self.deltaZ = DeltaZ
        self.contador = 0
        self.limite = Limite
        if Estado == 0:
            self.zona = 1
            self.tiempoZ = TiempoActual + DeltaZ
        else:
            self.zona = 0
            self.tiempoZ = limite
            
    def FinDeServicio(self,Cola):
        tiempoA = self.tiempoFS
        self.estado = 0
        if Cola.q > 0:
            Cola.q -=1
            self.zona = 1
            self.tiempoZ = tiempoA + self.deltaZ
        else:
            self.zona = 0
        self.tiempoFS = self.limite
        return tiempoA
    
    def LlegadaPS(self):
        tiempoA = self.tiempoZ
        self.tiempoFS = tiempoA
        self.zona = 0
        self.estado = 1
        self.tiempoFS += self.deltaFS[self.contador]
        self.contador += 1
        self.tiempoZ = self.limite
        return tiempoA
    
    def ReinicioContador(self):
        if self.contador > len(self.deltaFS)-1:
            self.contador = 0

##### Función para imprimir Tiempo, recibe variables en el siguiente orden: Hora Actual, LLegada del Cliente, Fin de Servicio, N° Cola (q) y Estado del Puesto de Servicio(PS)'

In [150]:
def imprimirTiempo(Cola,PS,TiempoA):
    
    tiempoActual = transformarTiempo(TiempoA)
    tiempoLlegada = transformarTiempo(Cola.tiempoL)
    tiempoFinServicio = transformarTiempo(PS.tiempoFS)
    tiempoLlegadaZ = transformarTiempo(PS.tiempoZ)
    
    if PS.estado==0:
        tiempoFinServicio = "        "

    if PS.zona==0:
        tiempoLlegadaZ = "        "

    print(tiempoActual," | ",tiempoLlegada," | ",tiempoFinServicio," | ",tiempoLlegadaZ," | ",PS.zona," | ",Cola.q," | ",PS.estado)

##### Función vector de deltas de tiempo por segundo'

In [151]:
def delta(tiemposDelta):
    listaDelta = []
    for i in tiemposDelta:
        listaDelta.append(datetime.timedelta(seconds = i))
    return listaDelta

##### Función asignación de tiempo'

In [152]:
def tiempo(horas,minutos,segundos):
    tiempo = datetime.datetime.combine(datetime.datetime.today(),datetime.time(horas,minutos,segundos,0)) 
    return tiempo

In [153]:
def timedelta(i):
    return datetime.timedelta(seconds = i)

##### Declaración de los tiempos iniciales y los deltas

In [156]:
#Instancia del limite(HORA DE SERVICIO)'
limite = tiempo(12,0,0)

#Instancia de los tiempos iniciales
tiempoActual = tiempo(8,0,0)
tiempoL = tiempo(8,5,0)
tiempoFS = tiempo(8,3,0)

#Tiempos delta particulares del práctico
tiempoDL = [40]
tiempoDFS = [32]
deltaZ = timedelta(10)
deltaL = delta(tiempoDL)
deltaFS = delta(tiempoDFS)

#Instancia de la Cola y del Puesto de Servicio
cola = Cola(3,tiempoL,deltaL)
PS = PuestoDeServicio(1,tiempoFS,deltaFS,deltaZ,limite + deltaZ,tiempoActual)

##### Programa principal, un while que imprime toda la matriz.

In [157]:
print("  Actual  |  LLegada   | Fin de S.  | LlegadaPS  |  Z  |  q  |  PS |")
print("--------------------------------------------------------------------")

#Se imprime primero el estado inicial
imprimirTiempo(cola,PS,tiempoActual)


while tiempoActual < limite:
    
        if cola.tiempoL <= PS.tiempoFS  and cola.tiempoL <= PS.tiempoZ and cola.tiempoL < limite:
            tiempoActual= cola.Llegada(PS)
            imprimirTiempo(cola,PS,tiempoActual)

                  
            
        #Si el tiempo de fin de servicio es menor que el de llegada'
        elif PS.tiempoFS <= cola.tiempoL and PS.tiempoFS < limite and PS.estado == 1:
            tiempoActual = PS.FinDeServicio(cola)
            imprimirTiempo(cola,PS,tiempoActual)

            
        #Cierre
        elif PS.tiempoZ <= cola.tiempoL and PS.tiempoZ <= limite:
            tiempoActual = PS.LlegadaPS()
            imprimirTiempo(cola,PS,tiempoActual)

        
        else:
            if PS.estado == 1:
                print("El último cliente en el PS saldrá a las horas:",transformarTiempo(PS.tiempoFS))
            break
        
        #Reinicio de contadores
        cola.ReinicioContador()
        PS.ReinicioContador()

print("Clientes sin atención:",cola.q)
    

  Actual  |  LLegada   | Fin de S.  | LlegadaPS  |  Z  |  q  |  PS |
--------------------------------------------------------------------
08:00:00  |  08:05:00  |  08:03:00  |            |  0  |  3  |  1
08:03:00  |  08:05:00  |            |  08:03:10  |  1  |  2  |  0
08:03:10  |  08:05:00  |  08:03:42  |            |  0  |  2  |  1
08:03:42  |  08:05:00  |            |  08:03:52  |  1  |  1  |  0
08:03:52  |  08:05:00  |  08:04:24  |            |  0  |  1  |  1
08:04:24  |  08:05:00  |            |  08:04:34  |  1  |  0  |  0
08:04:34  |  08:05:00  |  08:05:06  |            |  0  |  0  |  1
08:05:00  |  08:05:40  |  08:05:06  |            |  0  |  1  |  1
08:05:06  |  08:05:40  |            |  08:05:16  |  1  |  0  |  0
08:05:16  |  08:05:40  |  08:05:48  |            |  0  |  0  |  1
08:05:40  |  08:06:20  |  08:05:48  |            |  0  |  1  |  1
08:05:48  |  08:06:20  |            |  08:05:58  |  1  |  0  |  0
08:05:58  |  08:06:20  |  08:06:30  |            |  0  |  0  |  1
08:0

08:43:00  |  08:43:40  |            |  08:43:04  |  1  |  3  |  0
08:43:04  |  08:43:40  |  08:43:36  |            |  0  |  3  |  1
08:43:36  |  08:43:40  |            |  08:43:46  |  1  |  2  |  0
08:43:40  |  08:44:20  |            |  08:43:46  |  1  |  3  |  0
08:43:46  |  08:44:20  |  08:44:18  |            |  0  |  3  |  1
08:44:18  |  08:44:20  |            |  08:44:28  |  1  |  2  |  0
08:44:20  |  08:45:00  |            |  08:44:28  |  1  |  3  |  0
08:44:28  |  08:45:00  |  08:45:00  |            |  0  |  3  |  1
08:45:00  |  08:45:40  |  08:45:00  |            |  0  |  4  |  1
08:45:00  |  08:45:40  |            |  08:45:10  |  1  |  3  |  0
08:45:10  |  08:45:40  |  08:45:42  |            |  0  |  3  |  1
08:45:40  |  08:46:20  |  08:45:42  |            |  0  |  4  |  1
08:45:42  |  08:46:20  |            |  08:45:52  |  1  |  3  |  0
08:45:52  |  08:46:20  |  08:46:24  |            |  0  |  3  |  1
08:46:20  |  08:47:00  |  08:46:24  |            |  0  |  4  |  1
08:46:24  

09:18:20  |  09:19:00  |  09:18:36  |            |  0  |  6  |  1
09:18:36  |  09:19:00  |            |  09:18:46  |  1  |  5  |  0
09:18:46  |  09:19:00  |  09:19:18  |            |  0  |  5  |  1
09:19:00  |  09:19:40  |  09:19:18  |            |  0  |  6  |  1
09:19:18  |  09:19:40  |            |  09:19:28  |  1  |  5  |  0
09:19:28  |  09:19:40  |  09:20:00  |            |  0  |  5  |  1
09:19:40  |  09:20:20  |  09:20:00  |            |  0  |  6  |  1
09:20:00  |  09:20:20  |            |  09:20:10  |  1  |  5  |  0
09:20:10  |  09:20:20  |  09:20:42  |            |  0  |  5  |  1
09:20:20  |  09:21:00  |  09:20:42  |            |  0  |  6  |  1
09:20:42  |  09:21:00  |            |  09:20:52  |  1  |  5  |  0
09:20:52  |  09:21:00  |  09:21:24  |            |  0  |  5  |  1
09:21:00  |  09:21:40  |  09:21:24  |            |  0  |  6  |  1
09:21:24  |  09:21:40  |            |  09:21:34  |  1  |  5  |  0
09:21:34  |  09:21:40  |  09:22:06  |            |  0  |  5  |  1
09:21:40  

09:53:40  |  09:54:20  |            |  09:53:46  |  1  |  8  |  0
09:53:46  |  09:54:20  |  09:54:18  |            |  0  |  8  |  1
09:54:18  |  09:54:20  |            |  09:54:28  |  1  |  7  |  0
09:54:20  |  09:55:00  |            |  09:54:28  |  1  |  8  |  0
09:54:28  |  09:55:00  |  09:55:00  |            |  0  |  8  |  1
09:55:00  |  09:55:40  |  09:55:00  |            |  0  |  9  |  1
09:55:00  |  09:55:40  |            |  09:55:10  |  1  |  8  |  0
09:55:10  |  09:55:40  |  09:55:42  |            |  0  |  8  |  1
09:55:40  |  09:56:20  |  09:55:42  |            |  0  |  9  |  1
09:55:42  |  09:56:20  |            |  09:55:52  |  1  |  8  |  0
09:55:52  |  09:56:20  |  09:56:24  |            |  0  |  8  |  1
09:56:20  |  09:57:00  |  09:56:24  |            |  0  |  9  |  1
09:56:24  |  09:57:00  |            |  09:56:34  |  1  |  8  |  0
09:56:34  |  09:57:00  |  09:57:06  |            |  0  |  8  |  1
09:57:00  |  09:57:40  |  09:57:06  |            |  0  |  9  |  1
09:57:06  

10:55:22  |  10:55:40  |  10:55:54  |            |  0  |  12  |  1
10:55:40  |  10:56:20  |  10:55:54  |            |  0  |  13  |  1
10:55:54  |  10:56:20  |            |  10:56:04  |  1  |  12  |  0
10:56:04  |  10:56:20  |  10:56:36  |            |  0  |  12  |  1
10:56:20  |  10:57:00  |  10:56:36  |            |  0  |  13  |  1
10:56:36  |  10:57:00  |            |  10:56:46  |  1  |  12  |  0
10:56:46  |  10:57:00  |  10:57:18  |            |  0  |  12  |  1
10:57:00  |  10:57:40  |  10:57:18  |            |  0  |  13  |  1
10:57:18  |  10:57:40  |            |  10:57:28  |  1  |  12  |  0
10:57:28  |  10:57:40  |  10:58:00  |            |  0  |  12  |  1
10:57:40  |  10:58:20  |  10:58:00  |            |  0  |  13  |  1
10:58:00  |  10:58:20  |            |  10:58:10  |  1  |  12  |  0
10:58:10  |  10:58:20  |  10:58:42  |            |  0  |  12  |  1
10:58:20  |  10:59:00  |  10:58:42  |            |  0  |  13  |  1
10:58:42  |  10:59:00  |            |  10:58:52  |  1  |  12  

11:57:00  |  11:57:40  |  11:57:30  |            |  0  |  17  |  1
11:57:30  |  11:57:40  |            |  11:57:40  |  1  |  16  |  0
11:57:40  |  11:58:20  |            |  11:57:40  |  1  |  17  |  0
11:57:40  |  11:58:20  |  11:58:12  |            |  0  |  17  |  1
11:58:12  |  11:58:20  |            |  11:58:22  |  1  |  16  |  0
11:58:20  |  11:59:00  |            |  11:58:22  |  1  |  17  |  0
11:58:22  |  11:59:00  |  11:58:54  |            |  0  |  17  |  1
11:58:54  |  11:59:00  |            |  11:59:04  |  1  |  16  |  0
11:59:00  |  11:59:40  |            |  11:59:04  |  1  |  17  |  0
11:59:04  |  11:59:40  |  11:59:36  |            |  0  |  17  |  1
11:59:36  |  11:59:40  |            |  11:59:46  |  1  |  16  |  0
11:59:40  |  12:00:20  |            |  11:59:46  |  1  |  17  |  0
11:59:46  |  12:00:20  |  12:00:18  |            |  0  |  17  |  1
El último cliente en el PS saldrá a las horas: 12:00:18
Clientes sin atención: 17
